# **非構造化RAG**
非構造化または（準構造化）RAGは、テキスト、表、画像を組み合わせた文書を処理するために設計された手法です。テキストの分割による表の分断や、意味検索のための表の埋め込みの難しさといった課題に対応します。

ここでは、テキスト、表、画像を解析して分離するために、`unstructured.io`を使用しています。

**ツール参照:** [Unstructured](https://unstructured.io/)

---
## 環境構築 (macOS)

以下のツールを環境にインストールします。

```
brew install libmagic poppler libreoffice pandoc tesseract
```

`unstructured` のインストール:
*   [unstructured.io インストールガイド](https://docs.unstructured.io/open-source/installation/full-installation)

```
pip install "unstructured[all-docs]"
```

**対応ドキュメント形式:**

"csv", "doc", "docx", "epub", "image", "md", "msg", "odt", "org", "pdf", "ppt", "pptx", "rtf", "rst", "tsv", "xlsx"

---

## 特定のデータコネクタのインストール (例: S3):

```
pip install "unstructured-ingest[s3]" # 今回は不使用
```

**データコネクタとは:**

様々なデータソースからデータを取得し、アプリケーションやシステムで利用できるようにするためのツールまたはコンポーネントです。データの形式やアクセス方法の違いを吸収します。

**対応データコネクタ:**
"airtable", "azure", "azure-ai-search", "biomed", "box", "confluence", "couchbase", "delta-table", "discord", "dropbox", "elasticsearch", "gcs", "github", "gitlab", "google-drive", "jira", "mongodb", "notion", "opensearch", "onedrive", "outlook", "reddit", "s3", "sharepoint", "salesforce", "slack", "wikipedia"



In [ ]:
# !pip install -qU "unstructured[all-docs]" pdfplumber ipywidgets

# PDFから画像、テーブル、チャンク化テキストを読み込む

In [28]:
# load and extract images, tables, and chunk text
from unstructured.partition.pdf import partition_pdf
import os
# from .autonotebook import tqdm as notebook_tqdm

dataset_dir = "../data/pdf/"
# pdf_file_name = "57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf" # ページと図が多くテストに向いていないので変更
pdf_file_name = "Attention Is All You Need 1706.03762v7.pdf"
extract_image_block_output_dir = os.path.join("../data/", "test_images")
os.makedirs(extract_image_block_output_dir, exist_ok=True)

pdf_elements = partition_pdf(
    filename=os.path.join(dataset_dir, pdf_file_name),
#   chunking_strategy="by_title",
    extract_images_in_pdf=True,
    infer_table_structure=True,
    # max_characters=4000,
    # combine_text_under_n_chars=200,
    extract_image_block_output_dir=extract_image_block_output_dir, # defaultでは実行ファイルがあるフォルダ下にfiguresが作成される
)


## [unstructured.partition.pdf のパラメータ](https://docs.unstructured.io/open-source/core-functionality/partitioning)

### 主なパラメータ

| パラメータ | 説明 |
|---|---|
| `filename` | PDF ファイルのパスを指定します。 |
| `file` | PDF ファイルオブジェクトを指定します。`filename` または `file` のいずれかを指定する必要があります。 |
| `strategy` | PDF ファイルの分割戦略を指定します。 |
|  | `"auto"`: 自動的に最適な戦略を選択します (デフォルト)。 |
|  | `"fast"`: 高速な分割を行います。 |
|  | `"hi_res"`: 高解像度な分割を行います。 |
|  | `"ocr_only"`: OCR (光学文字認識) のみを行います。 |
| `infer_table_structure` | テーブル構造を推論するかどうかを指定します (`True`/`False`)。 |
| `ocr_languages` | OCR で使用する言語を指定します。 |
| `encoding` | ファイルのエンコーディングを指定します。 |
| `include_page_breaks` | ページ区切りを含めるかどうかを指定します (`True`/`False`)。 |
| `max_partition` | 最大分割数を指定します。 |
| `extract_image_block_output_dir` | 画像の出力フォルダを指定します。 |

### 詳細なパラメータ

| パラメータ | 説明 |
|---|---|
| `chunking_strategy` | 分割された要素をどのようにチャンクにまとめるかを指定します。 |
|  | `"by_title"`: タイトルごとにチャンクをまとめます。 |
|  | `"by_paragraph"`: 段落ごとにチャンクをまとめます。 |
|  | `"contiguous"`: 連続する要素をまとめてチャンクにします。 |
| `new_after_n_words` | チャンクの最大単語数を指定します。 |
| `new_after_n_pages` | チャンクの最大ページ数を指定します。 |
| `metadata_filename` | メタデータファイルを指定します。 |
| `pdf_text_extraction` | PDF テキスト抽出方法を指定します。 |
|  | `"textract"`: textract ライブラリを使用します。 |
|  | `"tika"`: Tika サーバーを使用します。 |
|  | `"paddlepaddle"`: PaddlePaddle OCR を使用します。 |

### パラメータの選択例

| 例 | 設定 |
|---|---|
| 高速な分割 | `strategy="fast"` |
| 高解像度な分割 | `strategy="hi_res"` |
| テーブル構造の推論 | `infer_table_structure=True` |
| 英語と日本語の OCR | `ocr_languages=["en", "ja"]` |
| ページ区切りを含める | `include_page_breaks=True` |

### 注意点

* パラメータの組み合わせによっては、期待通りの結果が得られない場合があります。
* PDF ファイルの内容や構造によっては、分割がうまくいかない場合があります。
* OCR を使用する場合、言語によっては精度が低い場合があります。

In [ ]:
# # check unique categories

from collections import Counter
category_counts = Counter(str(type(element)) for element in pdf_elements)
unique_categories = set(category_counts)
category_counts

In [ ]:
# extract unique types
unique_types = {el.to_dict()['type'] for el in pdf_elements}
unique_types

# chatGPTのAPI料金が高いので安いGoogle系に変更　（設定は面倒）

1. GCPの任意のプロジェクトでvertexAI APIを有効化[こちら](https://github.com/shohei0990/Multimodal_RAG_00/tree/main?tab=readme-ov-file)を参考にした。(私の場合は.envにAPIキーを記載)
2. ローカルマシンにgoogle-cloud-sdkインストール
- macOSの場合、Homebrewを使ってインストール
brew install google-cloud-sdk
1. 初期設定と認証
- 初期化コマンドを実行
gcloud init
- Googleアカウントでログイン
- プロジェクトを選択または新規作成
1. インストール確認
gcloud version  # バージョン確認
cloud auth list  # 認証状態確認
1. プロジェクト確認
gcloud projects list  # プロジェクト一覧表示
- 実行結果：
PROJECT_ID: my-project-xxxx-xxxx PROJECT_NUMBER: xxxxxxxx
1. リージョンとゾーンの設定
- 東京リージョンを設定
gcloud config set compute/region asia-northeast1
- 具体的なゾーンを設定
gcloud config set compute/zone asia-northeast1-a

これで、Google Cloud SDKのインストールから設定まで完了し、Notebookから利用できる状態になった

In [ ]:
# !pip install -qU google-cloud-aiplatform vertexai langchain_google_vertexai chromadb

## **🔸 Multimodal Embedding を使いたい場合（画像+テキスト）**
🔹 **現時点で Gemini AI Studio API ではマルチモーダルの埋め込みは未対応！**  
🔹 **GCP の Vertex AI API を使う必要があるのだ！**  

| **方法** | **Gemini AI Studio API** | **Vertex AI API (GCP)** |
|------------|-------------------|----------------|
| **テキスト埋め込み** | ✅ 可能（`models/embedding-001`） | ✅ 可能（`textembedding-gecko`） |
| **画像の埋め込み** | ❌ 未対応 | ❌ 未対応 |
| **画像 + テキスト埋め込み** | ❌ 直接は不可（画像→テキスト→埋め込み） | ❌ 直接は不可 |
| **商用利用** | ❌ 制限あり | ✅ 可能 |
| **おすすめ用途** | ✅ 個人開発 & プロトタイプ | ✅ 企業向け & 本番環境 |

🚀 **結論**  
🔹 **「Gemini AI Studio API」では、画像の埋め込みは直接できないが、画像 → テキスト → 埋め込み の方法なら可能！**  
🔹 **マルチモーダル埋め込みが必要なら、OpenAI CLIP など他の選択肢も検討するのだ！**


In [ ]:
# multi-modal-RAGにおいてgeminiAI/VertexAIのapiを使うにはGCPで(面倒な)設定をする必要がある様子。
# こちらを参考にした。https://github.com/shohei0990/Multimodal_RAG_00/tree/main?tab=readme-ov-file

import os
from dotenv import load_dotenv
load_dotenv()

import vertexai
import google.generativeai as genai

# 環境変数 GOOGLE_APPLICATION_CREDENTIALS を設定
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.expanduser("~/.config/gcloud/application_default_credentials.json") # たぶんgoogle-cloud-SDKを"~"にインストールしたのでこのファイルがここにできたのだと思う。
vertexai.init(project=os.getenv("gcp_project_id"), location="us-central1")
genai.configure(api_key=os.getenv("VERTEXAI_API_KEY"))

# for m in genai.list_models():
#   if 'generateContent' in m.supported_generation_methods:
#     print(m.name)

In [46]:
# checking api, とりあえずレスポンスが文字で出力されるモデルでテスト、画像の埋め込みはあとで"textembedding-gecko-multimodal"に変える

llm = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.2,
    }
)

# 画像の処理例
from PIL import Image
image_path = "../data/test_images/figure-4-2.jpg"
image = Image.open(image_path)
response = llm.generate_content(
    ["画像の説明を日本語でしてください", image]  # image_dataはPILImage, バイト列, または画像のパスです
)
print(response.text)

図は、スケール付きドット積注意機構（Scaled Dot-Product Attention）とマルチヘッド注意機構（Multi-Head Attention）のアーキテクチャを示しています。

**左側の図：スケール付きドット積注意機構**

この図は、スケール付きドット積注意機構の処理手順を段階的に示しています。

1. **Q, K, V:** クエリ（Q）、キー（K）、バリュー（V）の3つの行列が入力されます。これらは、通常、入力シーケンスのエンコーディング表現です。
2. **MatMul:** Q と K の行列積を計算します。これは、クエリとキー間の類似度を計算するステップです。
3. **Scale:** 行列積をスケール係数で除算します。これは、大きな値によるSoftmax関数の勾配消失問題を防ぐために行われます。
4. **Mask (opt.):** 必要に応じて、マスクを適用します。これは、シーケンスの将来の情報を使用しないようにする際に用いられます（例：自己回帰モデル）。
5. **SoftMax:** 行列の各要素を確率に変換します。これは、各クエリに対するキーの重み付けを行います。
6. **MatMul:** 重み付けされたキー（SoftMaxの結果）とVの行列積を計算します。これにより、クエリに対するコンテキストベクトルが生成されます。


**右側の図：マルチヘッド注意機構**

この図は、複数のスケール付きドット積注意機構を並列に実行するマルチヘッド注意機構を示しています。

1. **Linear:** 入力（V, K, Q）を線形変換します。これは、異なる特徴空間で注意機構を実行するために行われます。
2. **Scaled Dot-Product Attention:** 複数の（図では3つ）スケール付きドット積注意機構を並列に実行します。各機構は、異なる線形変換された入力を使用します。
3. **Concat:** 各ヘッドからの出力を連結します。
4. **Linear:** 連結された出力を線形変換して、最終的な出力ベクトルを生成します。


要約すると、マルチヘッド注意機構は、複数のスケール付きドット積注意機構を組み合わせることで、入力シーケンスの様々な側面を捉え、より表現力豊かな注意機構を実現しています。  `h

In [ ]:
# !gcloud auth application-default login # 最初の一回だけ必要っぽい

In [ ]:
# ! pip install -qU langchain_chroma

1.PDFドキュメントのロード: UnstructuredPDFLoaderを使用して、PDFドキュメントをロードし、テキスト、画像、テーブルなどの要素を抽出します。
2.画像の説明文の生成: Gemini 1.5 Flashを使用して、画像の説明文を生成します。
3.ChromaDBへの登録: テキスト、画像の説明文、テーブルの内容をChromaDBに登録します。テキストの埋め込みには、textembedding-gecko-multimodalを使用します。私の環境では1.5h以上がかかりました。

In [60]:
import vertexai
import google.generativeai as genai
from PIL import Image
from typing import List

from langchain_google_vertexai import VertexAI, VertexAIEmbeddings
from langchain_chroma import Chroma
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_core.documents import Document
from langchain_community.vectorstores.utils import filter_complex_metadata


llm = VertexAI(model_name="gemini-1.5-flash", temperature=0.2, max_output_tokens=2048)
# text_embeddings = VertexAIEmbeddings(model_name="multimodalembedding")
text_embeddings = VertexAIEmbeddings(model_name="textembedding-gecko")

# ドキュメントのロード
pdf_path = "../data/pdf/Attention Is All You Need 1706.03762v7.pdf"
loader = UnstructuredPDFLoader(pdf_path, mode="elements", strategy="fast") # 前のセルでテストしたpartition_pdf関数がloader.load()の中で間接的に使用されています。
documents = loader.load()

# テキストの分割
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# 画像の説明文を生成
def generate_image_descriptions(documents: List[Document]) -> List[str]:
    image_descriptions = []
    for doc in documents:
        if "image_path" in doc.metadata:
            image = Image.open(doc.metadata["image_path"])
            response = llm.generate_content(
                [f"この画像を詳しく説明してください。重要な特徴や要素を含めて説明してください。", image]
            )
            image_descriptions.append(response.text)
        else:
            image_descriptions.append("")
    return image_descriptions

image_descriptions = generate_image_descriptions(documents)

# ChromaDBへの登録 (テキストと画像の説明文を組み合わせる)
metadatas = [doc.metadata for doc in documents]

# メタデータから複雑な型を削除
# ドキュメントリストに対してfilter_complex_metadataを適用
filtered_metadatas = []
for doc in documents:
    filtered_metadata = filter_complex_metadata([doc])  # ドキュメントをリストで渡す
    filtered_metadatas.append(filtered_metadata[0].metadata if filtered_metadata else {})  # 結果からメタデータを取り出す

metadatas = filtered_metadatas

combined_texts = []
for i, doc in enumerate(documents):
    if doc.metadata.get("type") == "image":
        combined_texts.append(f"{doc.page_content}\n画像の説明: {image_descriptions[i]}")
    elif doc.metadata.get("type") == "table":
        # テーブルの要約文を生成するコードを追加 (例: LLMを使用)
        table_summary = f"テーブルの内容: {doc.page_content}"  # プレースホルダー
        combined_texts.append(f"{doc.page_content}\n{table_summary}")
    else:
        combined_texts.append(doc.page_content)

db = Chroma.from_texts(combined_texts, text_embeddings, metadatas=metadatas, persist_directory="../data/chroma_db")
# db.persist()

クォータとは、クラウドサービスなどのリソースの使用量に対する制限のことです。

クォータの目的:

リソースの公平な分配: 全てのユーザーが公平にリソースを利用できるようにするため。
システムの安定性維持: 特定のユーザーによる過剰なリソース消費を防ぎ、システム全体の安定性を維持するため。
コスト管理: 意図しない過剰なリソース消費による高額な請求を防ぐため。
セキュリティ: 悪意のあるユーザーによるリソースの不正利用を防ぐため。
クォータの種類:

クォータには、以下のような種類があります。

時間あたりのリクエスト数: 1分あたり、1時間あたりなどのリクエスト数制限。
ストレージ容量: データの保存容量制限。
コンピューティングリソース: CPU、メモリなどの使用量制限。
ネットワーク帯域幅: データ転送量制限。

In [65]:
import pandas as pd
import time

# 質問応答: ユーザーからの質問を受け取り、ChromaDBから関連する情報を検索し、Gemini 1.5 Flashを使用して回答を生成します。
queries = [
    "Transformerモデルの主要な特徴は何ですか？",
    "Attention Is All You Needの著者",
    "TransformerモデルのAttention機構について説明してください。",
    "Transformerモデルは、RNNやCNNと比較してどのような利点がありますか？",
    "Transformerモデルの応用例をいくつか挙げてください。",
    "Transformerモデルの課題は何ですか？",
    "この論文で提案されているTransformerモデルの計算コストについて説明してください。"
]
questions = []
answers = []
contexts = []

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

for query in queries:
    result = qa({"query": query})
    questions.append(query)
    answers.append(result["result"])
    contexts.append(result["source_documents"])
    time.sleep(10)  # クォータ制限の上限に引っかかるので各クエリの後に待機 

df = pd.DataFrame({"question": questions, "response": answers, "context": contexts})
print(df)

                                        question  \
0                     Transformerモデルの主要な特徴は何ですか？   
1                   Attention Is All You Needの著者   
2        TransformerモデルのAttention機構について説明してください。   
3     Transformerモデルは、RNNやCNNと比較してどのような利点がありますか？   
4                Transformerモデルの応用例をいくつか挙げてください。   
5                        Transformerモデルの課題は何ですか？   
6  この論文で提案されているTransformerモデルの計算コストについて説明してください。   

                                            response  \
0  Transformerモデルの主要な特徴は、**自己注意メカニズム**です。これは、入力シー...   
1  The authors of "Attention Is All You Need" are...   
2  Transformerモデルは、マルチヘッドアテンションを3つの異なる方法で使用します。 \...   
3  Transformerモデルは、RNNやCNNと比較して、以下の利点があります。\n\n* ...   
4                 Transformerモデルの応用例をいくつか挙げてください。 \n   
5  I don't know. I need more information about th...   
6  I don't know. I need more context to answer th...   

                                             context  
0  [page_content='Figure 1: The Transformer - mod...  
1  [page_content='Attent

In [ ]:
import pandas as pd
from langchain.evaluation import load_evaluator
from langchain.evaluation import EvaluatorType

eval_llm = VertexAI(model_name="gemini-2.0-flash", temperature=0.0, max_output_tokens=2048)

relevance_evaluator = load_evaluator(
    EvaluatorType.QA,
    criteria="relevance",
    model="gemini-2.0-flash",
    model_kwargs={"temperature": 0}
)

accuracy_evaluator = load_evaluator(
    EvaluatorType.QA,
    criteria="accuracy",
    model="gemini-2.0-flash",
    model_kwargs={"temperature": 0}
)

def generate_synthetic_answer(question):
    """LLMを使用して質問に対する正解データを生成します."""
    prompt = f"質問: {question}\n 正解データ:"
    response = eval_llm.invoke(prompt).strip()
    return response.strip()

def evaluate_response(df):
    """
    与えられたデータリストに対する Relevance と Accuracy の評価を行い、結果を DataFrame で返します。
    """
    def _flatten_context(context_val):
        if isinstance(context_val, list):
            return "\n".join([item.page_content if hasattr(item, "page_content") else str(item) for item in context_val])
        return str(context_val)
    
    df["ContextText"] = df["context"].apply(lambda x: _flatten_context(x))
    df["target_answer"] = df["question"].apply(generate_synthetic_answer) # 疑似的な正解データを生成

    def _evaluate_row(row):
        relevance_score = relevance_evaluator.evaluate_strings(
            prediction=row["response"],
            input=row["question"],
            reference=row["ContextText"]
        )
        accuracy_score = accuracy_evaluator.evaluate_strings(
            prediction=row["response"],
            input=row["question"],
            reference=row["target_answer"]
        )
        return relevance_score, accuracy_score
    
    df[["Relevance", "Accuracy"]] = df.apply(lambda row: pd.Series(_evaluate_row(row)), axis=1)
    
    df["RelevanceScore"] = df["Relevance"].apply(lambda x: 1 if x["value"].upper() == "CORRECT" else 0)
    df["AccuracyScore"] = df["Accuracy"].apply(lambda x: 1 if x["value"].upper() == "CORRECT" else 0)
    
    # 平均スコアを計算
    avg_relevance = df["RelevanceScore"].mean()
    avg_accuracy = df["AccuracyScore"].mean()
    
    print(f"Average Relevance: {avg_relevance:.2f}")
    print(f"Average Accuracy: {avg_accuracy:.2f}")

    # きれいに処理したContextTextカラムがあるので、その前段階のcontextカラムを削除して返す
    return df.drop(columns=['context'])

In [71]:
results = evaluate_response(df)
results 

/var/folders/yl/5c7mgh4j7j9clvvpsvwjbvhr0000gn/T/ipykernel_3764/3677457431.py:24: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = eval_llm(prompt)  # eval_llmを使用


Average Relevance: 0.29
Average Accuracy: 0.43


,question,response,ContextText,target_answer,Relevance,Accuracy,RelevanceScore,AccuracyScore
0,Transformerモデルの主要な特徴は何ですか？,Transformerモデルの主要な特徴は、**自己注意メカニズム**です。これは、入力シー...,Figure 1: The Transformer - model architecture...,Transformerモデルの主要な特徴は以下の通りです。\n\n* **Attenti...,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0
1,Attention Is All You Needの著者,"The authors of ""Attention Is All You Need"" are...",Attention Is All You Need\nAttention Is All Yo...,"Ashish Vaswani, Noam Shazeer, Niki Parmar, Jak...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1
2,TransformerモデルのAttention機構について説明してください。,Transformerモデルは、マルチヘッドアテンションを3つの異なる方法で使用します。 \...,The Transformer uses multi-head attention in t...,TransformerモデルのAttention機構は、入力シーケンス内の各単語（またはトー...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1
3,Transformerモデルは、RNNやCNNと比較してどのような利点がありますか？,Transformerモデルは、RNNやCNNと比較して、以下の利点があります。\n\n* ...,ByteNet [18] Deep-Att + PosUnk [39] GNMT + RL ...,Transformerモデルは、RNNやCNNと比較して、以下の点で大きな利点があります。\...,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",0,1
4,Transformerモデルの応用例をいくつか挙げてください。,Transformerモデルの応用例をいくつか挙げてください。 \n,√\n√\n√,Transformerモデルは、その並列処理能力と注意機構により、様々な分野で優れた性能を発...,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0
5,Transformerモデルの課題は何ですか？,I don't know. I need more information about th...,Figure 1: The Transformer - model architecture...,Transformerモデルは非常に強力ですが、いくつかの課題も抱えています。主な課題として...,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0
6,この論文で提案されているTransformerモデルの計算コストについて説明してください。,I don't know. I need more context to answer th...,√\n√\n√,この論文で提案されているTransformerモデルの計算コストは、主に以下の2つの要因によ...,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0


# memo: llm.generate_content と llm.invoke の違い

llmの定義
from langchain_google_vertexai import VertexAI
llm = VertexAI(model_name="xxx", temperature=yyy, max_output_tokens=zzz)


| 特徴             | llm.generate_content                               | llm.invoke                                     |
| ---------------- | ----------------------------------------------------- | ------------------------------------------------- |
| 役割             | 汎用的なコンテンツ生成                                | シンプルなテキスト生成                               |
| 入力             | テキスト、画像など                                    | テキスト                                           |
| 主な用途         | マルチモーダルモデル (例: Gemini)                       | テキストベースの LLM (例: GPT-3)                      |
| 出力             | GenerationResponse オブジェクト                       | 生成されたテキスト                                 |
| 複雑さ           | 低レベル、より細かい制御が可能                           | 高レベル、使いやすい                               |